In [6]:
# pip uninstall kafka-python

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.streaming import StreamingContext
from kafka import KafkaProducer
from kafka import KafkaConsumer, TopicPartition
import pyarrow.parquet as pq
import time

In [2]:
# Initialize Kafka consumer
# consumer = KafkaConsumer('log_topic', bootstrap_servers='localhost:9092')

# Initialize Spark session
spark = SparkSession.builder.appName('LogProcessing').getOrCreate()

# Initialize StreamingContext with batch interval of 10 seconds
ssc = StreamingContext(spark.sparkContext, 10)


23/05/30 18:06:18 WARN Utils: Your hostname, Tanmays-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.31.18.59 instead (on interface en0)
23/05/30 18:06:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/30 18:06:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/tanmaysingla/opt/anaconda3/lib/python3.9/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [3]:
schema = "value STRING"
dataframe = spark.createDataFrame([], schema)

# Set Kafka broker(s) and topic
bootstrap_servers = 'localhost:9092'
topic = 'log_topic_42'
group_id = 'consumer_group'

# Initialize Kafka producer
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

consumer = KafkaConsumer(
#     topic,
    bootstrap_servers=bootstrap_servers,
    group_id=group_id,
    auto_offset_reset='earliest',
    enable_auto_commit=False
)
consumer.assign([TopicPartition(topic, 0)])

In [4]:
# Read log file
log_file_path = '42MBSmallServerLog.log'
f = open(log_file_path, 'r')
count = 0
for line in f:
#     if count < 5:
    producer.send(topic, value=line.strip().encode('utf-8'))
#         count += 1
    
producer.send(topic, value="end_of_file".encode('utf-8'))
producer.close()

In [5]:
flag = False
# Continuously poll for new messages every 10 seconds
while True:
    
    # Poll Kafka for new messages
    messages = consumer.poll(timeout_ms=10000)

    if messages:
        for partition, message_list in messages.items():
            for message in message_list:
                # Process the received message
                value = message.value.decode('utf-8')
                if value == "end_of_file":
                    flag = True
                    break
                print(f'Received message: {value}')
                # Convert the value into a DataFrame row
                row = [value]
                row_df = spark.createDataFrame([row], schema)

                # Append the new row to the existing DataFrame
                dataframe = dataframe.union(row_df)

        # Commit the offset to mark messages as processed
        consumer.commit()
#         dataframe.show(10, truncate=False)
#         dataframe.count()
    if flag:
        consumer.close()
        break
    # Wait for 10 seconds before polling again
#     time.sleep(10)



Received message: 10.82.240.107 - - [26/Dec/2118:12:00:00 +0530] "GET /Archives/edgar/data/0001411207/000156459021052445/0001564590-21-052445-index.htm HTTP/1.0" 304 1638
Received message: 169.174.119.238 - - [26/Dec/2118:12:00:00 +0530] "DELETE /Archives/edgar/data/0000004281/000119312517062657/R97.htm HTTP/1.0" 303 40118
Received message: 116.82.220.168 - - [26/Dec/2118:12:00:00 +0530] "POST /Archives/edgar/data/0001168220/000116822010000017/form10k_123109.htm HTTP/1.0" 403 5856
Received message: 99.149.129.109 - - [26/Dec/2118:12:00:00 +0530] "POST /Archives/edgar/data/0001765880/000121390021017179/s131103_10k.htm HTTP/1.0" 500 25206
Received message: 167.134.232.8 - - [26/Dec/2118:12:00:00 +0530] "PUT /Archives/edgar/data/1537583/999999999720003444/9999999997-20-003444-index.htm HTTP/1.0" 200 29727
Received message: 128.171.81.194 - - [26/Dec/2118:12:00:00 +0530] "GET /Archives/edgar/data/0001491487/000114420415051928/0001144204-15-051928.txt HTTP/1.0" 304 3695
Received message: 74

Received message: 113.105.153.46 - - [26/Dec/2118:12:00:00 +0530] "GET /Archives/edgar/data/0001729361/000121390021017176/s131100_10k.htm HTTP/1.0" 404 37220
Received message: 64.138.153.214 - - [26/Dec/2118:12:00:00 +0530] "POST /Archives/edgar/data/0001760283/000121390021017155/s131124_10k.htm HTTP/1.0" 200 28991
Received message: 122.255.156.225 - - [26/Dec/2118:12:00:00 +0530] "POST /Archives/edgar/data/0001491487/000114420415051928/0001144204-15-051928.txt HTTP/1.0" 403 3530
Received message: 131.168.191.201 - - [26/Dec/2118:12:00:00 +0530] "POST /Archives/edgar/data/0000752177/000093247112005362/0000932471-12-005362.txt HTTP/1.0" 200 11224
Received message: 59.17.165.90 - - [26/Dec/2118:12:00:00 +0530] "GET /Archives/edgar/data/0001260125/000114420419015093/0001144204-19-015093.txt HTTP/1.0" 502 45230
Received message: 121.105.234.91 - - [26/Dec/2118:12:00:00 +0530] "PUT /Archives/edgar/data/0001439124/000129281422000788/ HTTP/1.0" 500 18451
Received message: 117.53.227.87 - - [2

Received message: 202.110.83.13 - - [26/Dec/2118:12:00:00 +0530] "DELETE /Archives/edgar/data/0001439124/000129281422000790/ HTTP/1.0" 502 18993
Received message: 150.150.147.220 - - [26/Dec/2118:12:00:00 +0530] "POST /Archives/edgar/data/0001035018/000119312507272779/g47607g86j71.jpg HTTP/1.0" 200 55811
Received message: 80.137.207.210 - - [26/Dec/2118:12:00:00 +0530] "DELETE /Archives/edgar/data/0001179929/000117992923000043/moh-20230320_g29.jpg HTTP/1.0" 200 50545
Received message: 178.129.95.168 - - [26/Dec/2118:12:00:00 +0530] "DELETE /Archives/edgar/data/0001737706/000119312523067666/d466377d6k.htm HTTP/1.0" 404 11163
Received message: 8.102.44.67 - - [26/Dec/2118:12:00:00 +0530] "PUT /Archives/edgar/data/0000205007/000114554923005581/0001145549-23-005581.txt HTTP/1.0" 200 11025
Received message: 21.57.238.201 - - [26/Dec/2118:12:00:00 +0530] "PUT /Archives/edgar/data/35315/000175272423067615/primary_doc.xml HTTP/1.0" 403 14738
Received message: 85.15.109.163 - - [26/Dec/2118:12:

Received message: 29.1.146.209 - - [26/Dec/2118:12:00:00 +0530] "GET /Archives/edgar/data/0001867102/000110465922086725/ HTTP/1.0" 404 9607
Received message: 197.124.85.179 - - [26/Dec/2118:12:00:00 +0530] "GET /Archives/edgar/data/0000205007/000114554923005577/0001145549-23-005577.txt HTTP/1.0" 500 52813
Received message: 145.223.22.218 - - [26/Dec/2118:12:00:00 +0530] "POST /Archives/edgar/data/0000004281/000119312517062657/R99.htm HTTP/1.0" 304 44532
Received message: 221.165.62.148 - - [26/Dec/2118:12:00:00 +0530] "POST /Archives/edgar/data/0000004281/000119312517062657/R94.htm HTTP/1.0" 404 40975
Received message: 220.193.146.175 - - [26/Dec/2118:12:00:00 +0530] "DELETE /Archives/edgar/data/0001518715/000151871522000011/0001518715-22-000011-index.htm HTTP/1.0" 500 7217
Received message: 184.230.214.105 - - [26/Dec/2118:12:00:00 +0530] "POST /Archives/edgar/data/0001002638/000100263815000016/0001002638-15-000016-index.htm HTTP/1.0" 500 8280
Received message: 66.107.197.119 - - [26/

Received message: 83.137.99.154 - - [26/Dec/2118:12:00:00 +0530] "PUT /Archives/edgar/data/0001468457/000093247112005362/0000932471-12-005362.txt HTTP/1.0" 303 55046
Received message: 178.60.152.125 - - [26/Dec/2118:12:00:00 +0530] "POST /Archives/edgar/data/0001260125/000114420419015093/tv516458_10k.htm HTTP/1.0" 500 26005
Received message: 86.202.36.241 - - [26/Dec/2118:12:00:00 +0530] "PUT /Archives/edgar/data/0001179929/000117992923000043/moh-20230320_g28.jpg HTTP/1.0" 403 28790
Received message: 191.94.250.208 - - [26/Dec/2118:12:00:00 +0530] "GET /Archives/edgar/data/0001787168/000120919122010338/0001209191-22-010338.txt HTTP/1.0" 303 46797
Received message: 89.222.234.60 - - [26/Dec/2118:12:00:00 +0530] "GET /Archives/edgar/data/0001179929/000117992922000054/a2022definitiveproxy.htm HTTP/1.0" 200 18069
Received message: 36.214.2.108 - - [26/Dec/2118:12:00:00 +0530] "GET /Archives/edgar/data/0001439124/000129281422000790/ HTTP/1.0" 500 38332
Received message: 33.39.83.229 - - [26

...

Received message: 165.158.254.42 - - [26/Dec/2118:12:00:00 +0530] "GET /Archives/edgar/data/0001267332/000117494721000300/form10k-25692_indexpl20031.htm HTTP/1.0" 502 45859
Received message: 92.239.120.134 - - [26/Dec/2118:12:00:00 +0530] "GET /Archives/edgar/data/0001608293/000119312514225607/g732497g91g62.jpg HTTP/1.0" 200 58978
Received message: 167.215.133.37 - - [26/Dec/2118:12:00:00 +0530] "PUT /Archives/edgar/data/0001179929/000117992923000043/moh-20230320_g29.jpg HTTP/1.0" 200 39261
Received message: 79.175.52.116 - - [26/Dec/2118:12:00:00 +0530] "GET /Archives/edgar/data/000132756719000032/0001327567-19-000032-index.html HTTP/1.0" 304 5065
Received message: 143.140.179.161 - - [26/Dec/2118:12:00:00 +0530] "PUT /Archives/edgar/data/0001047862/000104786222000172/xslF345X03/edgar.xml HTTP/1.0" 500 10108
Received message: 160.2.236.89 - - [26/Dec/2118:12:00:00 +0530] "POST /Archives/edgar/data/0001047862/000104786222000172/xslF345X03/edgar.xml HTTP/1.0" 303 57384
Received message:

Received message: 60.186.22.31 - - [26/Dec/2118:12:00:00 +0530] "PUT /Archives/edgar/data/0000004281/000119312517062657/R98.htm HTTP/1.0" 303 52671
Received message: 6.105.155.103 - - [26/Dec/2118:12:00:00 +0530] "DELETE /Archives/edgar/data/0001852440/000000000022002912/filename2.txt HTTP/1.0" 304 3554
Received message: 179.252.27.100 - - [26/Dec/2118:12:00:00 +0530] "POST /Archives/edgar/data/0001801390/000121390021017183/s131106_10k.htm HTTP/1.0" 303 13378
Received message: 175.251.72.243 - - [26/Dec/2118:12:00:00 +0530] "DELETE /Archives/edgar/data/0001867102/000110465922080650/ HTTP/1.0" 304 14196
Received message: 99.0.52.236 - - [26/Dec/2118:12:00:00 +0530] "PUT /Archives/edgar/data/0000101382/000156459018002843/umbf-10k_20171231.htm HTTP/1.0" 303 37065
Received message: 207.228.92.128 - - [26/Dec/2118:12:00:00 +0530] "PUT /Archives/edgar/data/0000004281/000119312517062657/R94.htm HTTP/1.0" 502 52125
Received message: 197.1.230.185 - - [26/Dec/2118:12:00:00 +0530] "GET /Archives

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/tanmaysingla/opt/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/tanmaysingla/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/tanmaysingla/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


Received message: 76.18.64.182 - - [26/Dec/2118:12:00:00 +0530] "POST /Archives/edgar/data/0001439124/000129281422000790/ HTTP/1.0" 403 13395


KeyboardInterrupt: 

In [6]:
dataframe.show(10, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                   |
+--------------------------------------------------------------------------------------------------------------------------------------------------------+
|10.82.240.107 - - [26/Dec/2118:12:00:00 +0530] "GET /Archives/edgar/data/0001411207/000156459021052445/0001564590-21-052445-index.htm HTTP/1.0" 304 1638|
|169.174.119.238 - - [26/Dec/2118:12:00:00 +0530] "DELETE /Archives/edgar/data/0000004281/000119312517062657/R97.htm HTTP/1.0" 303 40118                 |
|116.82.220.168 - - [26/Dec/2118:12:00:00 +0530] "POST /Archives/edgar/data/0001168220/000116822010000017/form10k_123109.htm HTTP/1.0" 403 5856          |
|99.149.129.109 - - [26/Dec/2118:12:00:00 +0530] "POST /Archives/edgar

In [ ]:
dataframe.count()

In [1]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from kafka import KafkaProducer

# Initialize Spark session
spark = SparkSession.builder.appName('LogProcessing').getOrCreate()

# Initialize StreamingContext with batch interval of 10 seconds
ssc = StreamingContext(spark.sparkContext, 10)

# Set Kafka broker(s) and topic
bootstrap_servers = 'localhost:9092'
topic = 'log_topic_42'
group_id = 'consumer_group'

# Initialize Kafka producer
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

# Read log file
log_file_path = '42MBSmallServerLog.log'
f = open(log_file_path, 'r')
count = 0
for line in f:
#     if count < 5:
    producer.send(topic, value=line.strip().encode('utf-8'))
#         count += 1
    
producer.send(topic, value="end_of_file".encode('utf-8'))
producer.close()

# Create a Kafka direct stream
kafka_params = {
    'bootstrap.servers': bootstrap_servers,
    'group.id': group_id,
    'auto.offset.reset': 'earliest'
}

kafka_stream = KafkaUtils.createDirectStream(
    ssc, [topic], kafkaParams=kafka_params
)

# Process the received messages
lines = kafka_stream.map(lambda x: x[1])

# Define your data processing logic here
def process_logs(rdd):
    if not rdd.isEmpty():
        # Convert RDD to DataFrame
        dataframe = spark.createDataFrame(rdd, 'string')

        # Perform transformations and EDA on the DataFrame
        # ... (Add your Spark operations here)
        dataframe.show()

# Apply the processing logic to each RDD in the stream
lines.foreachRDD(process_logs)

# Start the streaming context
ssc.start()

# Wait for the streaming to finish
ssc.awaitTermination()


ModuleNotFoundError: No module named 'pyspark.streaming.kafka'

In [2]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pykafka import KafkaClient

# Initialize Spark session
spark = SparkSession.builder.appName('LogProcessing').getOrCreate()

# Initialize StreamingContext with batch interval of 10 seconds
ssc = StreamingContext(spark.sparkContext, 10)

# Set Kafka broker(s) and topic
bootstrap_servers = 'localhost:9092'
topic = 'log_topic_42'
group_id = 'consumer_group'

# Create Kafka client
client = KafkaClient(hosts=bootstrap_servers)

# Get Kafka topic consumer
topic_consumer = client.topics[topic].get_simple_consumer(consumer_group=group_id,
                                                          auto_offset_reset=OffsetType.EARLIEST)

# Process the received messages
def process_logs(rdd):
    if not rdd.isEmpty():
        # Convert RDD to DataFrame
        dataframe = spark.createDataFrame(rdd, 'string')

        # Perform transformations and EDA on the DataFrame
        # ... (Add your Spark operations here)
        dataframe.show()

# Apply the processing logic to each RDD in the stream
lines = ssc.queueStream([])  # Create an empty DStream
lines.foreachRDD(process_logs)

# Start the streaming context
ssc.start()

# Read Kafka messages and add them to the DStream
for message in topic_consumer:
    if message is not None:
        lines = lines.union(ssc.sparkContext.parallelize([message.value.decode('utf-8')]))

# Wait for the streaming to finish
ssc.awaitTermination()


ModuleNotFoundError: No module named 'pykafka'